<a href="https://colab.research.google.com/github/dariadiatlova/TTS_HW/blob/master/HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание 1

В этом задании вам предстоит сделать так, чтобы TTS начал говорить другим голосом так, чтобы можно было контролировать интонации путем присвоения токена каждому слову.

**Важно!** Добавьте ссылку на свой форк с гитхаба в команду git clone

In [35]:
!rm -r -f /content/TTS_HW

In [36]:
!git clone https://github.com/dariadiatlova/TTS_HW.git
!cd TTS_HW && curl gdrive.sh | bash -s https://drive.google.com/file/d/1ubAsUdWqHFY3MZJZVLqzxjH6lR_hKRyz/view?usp=sharing 
!cd TTS_HW && tar -xf libri_training_data.tar.gz

Cloning into 'TTS_HW'...
remote: Enumerating objects: 292, done.
remote: Counting objects: 100% (292/292), done.
remote: Compressing objects: 100% (223/223), done.
remote: Total 292 (delta 81), reused 273 (delta 65), pack-reused 0
Receiving objects: 100% (292/292), 4.72 MiB | 16.45 MiB/s, done.
Resolving deltas: 100% (81/81), done.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2874  100  2874    0     0   9548      0 --:--:-- --:--:-- --:--:--  9548
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3287    0  3287    0     0  24901      0 --:--:-- --:--:-- --:--:-- 24901
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   

## Важно!

Следующая ячейка будет устанавливать библиотеку Apex, на колабе это может занять **12-20** минут! К сожалению, без этой библиотеки обучение данной реализации сети будет длиться слишком медленно, также из этой библиотеки используется оптимайзер FusedLamb, если же использовать стандартный Adam, обучение может стать нестабильным.

Если у вас есть своя локальная GPU, рекомендую использовать docker контейнер с Nvidia GPU Cloud - `nvcr.io/nvidia/pytorch:21.06-py3`, там эта библиотека уже предустановлена.

In [2]:
try:
  import apex
except Exception:
  ! git clone https://github.com/NVIDIA/apex.git
  % cd apex
  !pip install --target=$nb_path --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" .
  %cd ..

Cloning into 'apex'...
remote: Enumerating objects: 8717, done.
remote: Counting objects: 100% (1085/1085), done.
remote: Compressing objects: 100% (246/246), done.
remote: Total 8717 (delta 963), reused 839 (delta 839), pack-reused 7632
Receiving objects: 100% (8717/8717), 14.38 MiB | 20.00 MiB/s, done.
Resolving deltas: 100% (5959/5959), done.
/content/apex
/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:232: UserWarning: Disabling all use of wheels due to the use of --build-option / --global-option / --install-option.
  cmdoptions.check_install_build_global(options)
Processing /content/apex
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/

In [7]:
!pip install -r TTS_HW/requirements.txt
!pip install transformers
!pip install torchmetrics

  Cloning git://github.com/NVIDIA/dllogger.git (to revision 26a0f8f1958de2c0c460925ff6102a4d2486d6cc) to /tmp/pip-install-t5dqs1vg/dllogger_43ee1a5c8bff47a5a6c2207aa7f3576d
  Running command git clone -q git://github.com/NVIDIA/dllogger.git /tmp/pip-install-t5dqs1vg/dllogger_43ee1a5c8bff47a5a6c2207aa7f3576d
  Running command git rev-parse -q --verify 'sha^26a0f8f1958de2c0c460925ff6102a4d2486d6cc'
  Running command git fetch -q git://github.com/NVIDIA/dllogger.git 26a0f8f1958de2c0c460925ff6102a4d2486d6cc
  Running command git checkout -q 26a0f8f1958de2c0c460925ff6102a4d2486d6cc
     |████████████████████████████████| 183 kB 5.3 MB/s 
     |████████████████████████████████| 195 kB 38.6 MB/s 
  Created wheel for dllogger: filename=DLLogger-0.1.0-py3-none-any.whl size=5630 sha256=f41b4698557e8d1a229188f6f6a2e04aeedff6d21db4e2ff5dfe72f28ced15e5
  Stored in directory: /root/.cache/pip/wheels/69/76/a4/ae417dad079ab1cae58eb65601ec01d980c5e02915f4f1792f
  Created wheel for librosa: filename=lib

In [37]:
!cd TTS_HW && bash scripts/download_cmudict.sh
!cd TTS_HW && bash scripts/download_fastpitch.sh
!cd TTS_HW && bash scripts/download_waveglow.sh

Extracting nvidia_fastpitch_210824.pt ...
OK
Extracting nvidia_waveglow256pyt_fp16.pt ...
OK


### Описание задачи
Выше была склонирована оригинальная имплементация FastPitch [отсюда](https://github.com/NVIDIA/DeepLearningExamples/tree/master/PyTorch/SpeechSynthesis/FastPitch). В ней изменен лишь даталоадер - в батче помимо текста и целевой мелспектрограммы, также возвращаются токены `Prominencе`. Необходимо понять, как можно использовать эти токены и в какое место сети необходимо их добавить, чтобы затем, на инференсе, можно было контролировать интонации.

Так как мы сильно ограничены в ресурсах - обучать сеть с нуля не получится, необходимо лишь провести finetuning с оригинального чекпоинта от Nvidia, обученного на датасете LJSpeech. От вас требуется дообучить сеть на одном из голосов из датасета LibriTTS.

Все данные уже предобработаны и загружены, токены `Prominence` были извлечены с помощью [Wavelet Prosody Toolkit](https://github.com/asuni/wavelet_prosody_toolkit). Эти значения были квантизированы, в данных могут встречаться лишь значения 1, 2, 3. Эти значения извлечены для каждого слова и продублированы для каждой фонемы.

Из-за особенностей имплементации засовывать обучение в ноутбук крайне неудобно, поэтому предлагается изменить файлы исходного кода прямо открывая их в колабе (они видны в файл менеджере слева), а затем запустить обучение, выполнив ячейку ниже. Чтобы не потерять свои изменения, рекомендую, изменив файлы, просто коммитить их к себе в репозиторий прямо из колаба. Не забывайте это делать часто, так как колаб любит уничтожать вашу среду, даже если вы просто отошли попить кофе.

Уже 10 эпох обучения достаточно (около 15 минут обучения), чтобы была слышна разница. Mel loss должен опуститься где-то до значений 0.2-0.3, чтобы TTS звучал нормально. Так как данных очень мало, то если вы оставите сеть учиться надолго, скорее всего она переобучится, поэтому учить сильно дольше 20 эпох нет смысла.

In [39]:
!cd TTS_HW && bash scripts/train.sh



AMP=true, 1x5x2 (global batch size 10)

Traceback (most recent call last):
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py", line 148, in <module>
    from torch.distributed.run import get_args_parser, run
  File "/usr/local/lib/python3.7/dist-packages/torch/distributed/run.py", line 338, in <module>
    from torch.distributed.elastic.multiprocessing import Std
  File "/usr/local/lib/python3.7/dist-packages/torch/distributed/elastic/multiprocessing/__init__.py", line 69, in <module>
    from torch.distributed.elastic.multiprocessing.api import (  # noqa: F401
  File "/usr/local/lib/python3.7/dist-packages/torch/distributed/elastic/multiprocessing/api.py", line 25, in <module>
    from torch.distributed.elastic.multiprocessing.errors import ProcessFailure, record
  File "/usr/

После того как обучили сеть, прогоните ее на одной заданной фразе с разными значениями `Prominence`. Для этого в файле `inference.py` измените 384 строчку и запустите ячейку ниже. С разными значениями prominence фраза "What a day" должна произноситься с разной интонацией. Если вы этого добились, значит задание выполнено.

In [40]:
!cd TTS_HW && bash scripts/inference_example.sh


AMP=false, batch_size=32

DLL 2021-12-10 19:59:33.287503 - PARAMETER | input :  phrases/phrase_1_64.txt
DLL 2021-12-10 19:59:33.287618 - PARAMETER | output :  ./output/audio_phrase_1_64.txt
DLL 2021-12-10 19:59:33.287703 - PARAMETER | log_file :  ./output/audio_phrase_1_64.txt/nvlog_infer.json
DLL 2021-12-10 19:59:33.287783 - PARAMETER | save_mels :  False
DLL 2021-12-10 19:59:33.287868 - PARAMETER | cuda :  True
DLL 2021-12-10 19:59:33.287959 - PARAMETER | cudnn_benchmark :  True
DLL 2021-12-10 19:59:33.288046 - PARAMETER | fastpitch :  output/FastPitch_checkpoint_10.pt
DLL 2021-12-10 19:59:33.288129 - PARAMETER | waveglow :  pretrained_models/waveglow/nvidia_waveglow256pyt_fp16.pt
DLL 2021-12-10 19:59:33.288213 - PARAMETER | sigma_infer :  0.9
DLL 2021-12-10 19:59:33.288295 - PARAMETER | denoising_strength :  0.01
DLL 2021-12-10 19:59:33.288384 - PARAMETER | sampling_rate :  22050
DLL 2021-12-10 19:59:33.288498 - PARAMETER | stft_hop_length :  256
DLL 2021-12-10 19:59:33.288596 - PA

In [34]:
# prominance [1,1,1,1,1,1,3,3,3,3]
import IPython
IPython.display.Audio('/content/TTS_HW/output/audio_phrase_1_64.txt/audio_0.wav')

In [41]:
# prominance [3,3,3,3,3,3,1,1,1,1]
import IPython
IPython.display.Audio('/content/TTS_HW/output/audio_phrase_1_64.txt/audio_0.wav')

# Задание 2 (5 баллов)

Это хорошо, что мы научились контролировать интонацию в произносимых фразах, но что делать на инференсе, ведь у нас нет этих значений `Prominence`? Можно попробовать научиться предсказывать их из текста. Для этого можно обучить какую-нибудь LM, при этом учить с нуля также не имеет смысла.

В задании предлагается заполнить отсутствующие ячейки ниже. Необходимый для зачета F1 score на тестовой выборке должен быть около `0.75`. Не забудьте про маски!

In [ ]:
import torch
import transformers
import os
from pathlib import Path
import numpy as np
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForTokenClassification, BertTokenizer, BertConfig, BertModel
from tqdm import tqdm
import torchmetrics

device='cuda:0'

In [ ]:
def val_to_class(val):
    if val < 1:
        return 0
    if 1 < val < 2:
        return 1
    return 2

f1_criterion = torchmetrics.F1(num_classes=3)
acc_criterion = torchmetrics.Accuracy(num_classes=3)

class PromDataset(Dataset):
    def __init__(self, tokenizer, sentences, labels, max_len):
        self.len = len(sentences)
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        #####################
        ##### YOUR CODE #####
        #####################
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'tags': torch.tensor(label, dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

sentences = []
label_proms = []
label_boundaries = []
for filename in Path('TTS_HW/libri_training_data').glob('*.prom'):
    #####################
    ##### YOUR CODE #####
    #####################


tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
train_dataset = PromDataset(tokenizer, sentences[:-10], label_proms[:-10], 200)
test_dataset = PromDataset(tokenizer, sentences[-10:], label_proms[-10:], 200)

training_loader = DataLoader(train_dataset, batch_size=24, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=True)


model = ##### YOUR CODE #####
model.to(device)
optimizer = torch.optim.Adam(params =  model.parameters(), lr=2e-05)
for epoch in range(10):
    print(f"EPOCH:{epoch}")
    t_bar = tqdm(training_loader)
    model.train()
    for batch in t_bar:
        optimizer.zero_grad()
        #####################
        ##### YOUR CODE #####
        #####################
        t_bar.set_description(desc=f'loss; {loss.item()}')
        optimizer.step()

    model.eval()
    with torch.inference_mode():
        accs = []
        f1_scores = []

        for batch in test_loader:
            #####################
            ##### YOUR CODE #####
            #####################

    print(f"Accuracy: {torch.mean(torch.stack(accs)).item()}")
    print(f"F1 score: {torch.mean(torch.stack(f1_scores)).item()}")

Стоит отметить, что мы обучаем сеть на очень небольшом наборе данных - если же файнтюнить на полном ТТС датасете, то точность предсказаний меток может достичь 0.95, поэтому такую сеть можно использовать при реальном инференсе для более точного предсказания интонаций.

# Задание 3 (5 баллов) (опционально)
Если по какой-то причине что-то не получилось в предыдущих заданиях, то можно попробовать подготовить модель к инференсу. Задача - заэкспортировать код модели в ONNX, и проверить, что он работает.
Подсказка - вся модель скорее всего не заэкспортится сразу, стоит пробовать делать это по модулям, а если какая-то функция не поддерживается ONNX, то ее можно выделить отдельно и заэкспортировать, например, в TorchScript. На выходе ожидается, что есть набор onnx или torchscript модулей, выполнив последовательно которые можно получить тот же выход, что и при запуске модели на PyTorch.

Задание - по желанию, если не хочется делать что-то из того, что выше!